In [1]:
from retinaface import RetinaFace
import cv2
import matplotlib.pyplot as plt
from deepface import DeepFace
import numpy as np
import os
import ast

Face Pattern Recognition

In [ ]:
#Define image to work with. It represents the real classroom photo taken with the camera
classroomFolder=os.path.join("..","ImagesDeepface","RealClassroomPhotos")
#Select the model to be used for detection varying the list index
detectors = [ "mtcnn","opencv", "retinaface"]
#Change depending on the index to be evaluates
chosen_detector_index=2

m=1
boundingBoxDetectedFaces=[]

#Iterate through all real classroom photos
for file in os.listdir(classroomFolder):
    camera_local_file = os.path.join("..","ImagesDeepface","RealClassroomPhotos",file)
    boundingBoxDetectedFacesNthImage=[]
    image = cv2.imread(camera_local_file)
    faces = DeepFace.extract_faces(img_path=camera_local_file, detector_backend=detectors[chosen_detector_index],enforce_detection=False)
    #Obtain images dimensions
    image_height, image_width, _ = image.shape

    #Save the coordinates of all faces in order to paint it later
    for i in range(len(faces)):
        boundingBoxDetectedFacesNthImage.append((faces[i]["facial_area"]["x"],faces[i]["facial_area"]["y"],faces[i]["facial_area"]["w"],
                                                faces[i]["facial_area"]["h"]))
    # Draw bounding boxes for each detected face using the coordinates
    for element in boundingBoxDetectedFacesNthImage:
        left = element[0]
        top = element[1]
        width = element[2]
        height =  element[3]
        cv2.rectangle(
            image, 
            (left, top), 
            (left + width, top + height), 
            (0,0,200), 
            2  
        )
    boundingBoxDetectedFaces.append(boundingBoxDetectedFacesNthImage)
    # Print number of faces detected
    print(f"{len(boundingBoxDetectedFacesNthImage)} Faces were detected in {camera_local_file}")

    # Save the annotated image in the destinated folder
    output_path =os.path.join("..","ImagesDeepface","Detection",f"Detection{detectors[chosen_detector_index]}",f"faces_detected_{m}.jpeg")
    
    cv2.imwrite(output_path, image)
    #Increase the photo's index
    m=m+1


In [ ]:

def calculate_iou_detection(box1, box2):

    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    
    x_intersect_1 = max(x1, x2)
    y_intersect_1 = max(y1, y2)
    x_intersect_2 = min(x1 + w1, x2 + w2)
    y_intersect_2 = min(y1 + h1, y2 + h2)
    
    if x_intersect_2 <= x_intersect_1 or y_intersect_2 <= y_intersect_1:
        return 0.0
    
    area_intersect = (x_intersect_2 - x_intersect_1) * (y_intersect_2 - y_intersect_1)
    area_box1 = w1 * h1
    area_box2 = w2 * h2
    
    area_union = area_box1 + area_box2 - area_intersect
    
    iou = area_intersect / area_union
    
    return iou



In [2]:

with open(os.path.join("..","Data","groundTruth.txt"), "r") as file:
    content = file.read()

groundTruth=ast.literal_eval(content)




m=1
#The N-Element contains tuples of the N-Th image with informations of students to be recognized
tuples=[]
for element in groundTruth:
    i=1
    #(f"photo {m}-----------------------------------------")
    filteredgroundTruthObject=element[next(iter(element))]["regions"]
    tuplesnthPhoto=[]
    for element in filteredgroundTruthObject:
        #print(f"Person {i}-----")
        region=element["shape_attributes"]
        tuplesnthPhoto.append((region["x"],region["y"], region["width"], region["height"]))
        i=i+1
    tuples.append(tuplesnthPhoto)
    m=m+1

Compare detections and ground truth to construct the performance marks

In [ ]:
i=0
TPTotal=0
TNTotal=0
FPTotal=0
FNTotal=0

for element in tuples:
    TP=0
    TN=0
    FN=0
    FP=0
    verifiedPredictions=0    
    print(f"------Results for image {i+1}------")
    for face in element: #Compare each face of ground truth with the predictions' array. If found, add TP, if not identified, add FN
        found=False
        for detectedFace in boundingBoxDetectedFaces[i]:
            IoU=calculate_iou_detection(face, detectedFace)
            if (IoU>=0.5):
                found=True
                TP=TP+1
                verifiedPredictions=verifiedPredictions+1
                break
        if (found==False):
            FN=FN+1
    FP=FP+(len(boundingBoxDetectedFaces[i])-verifiedPredictions)
    print(F"TP: {TP}, TN:{TN}, FP:{FP} FN:{FN}")
    TPTotal=TPTotal+TP
    TNTotal=TNTotal+TN
    FPTotal=FPTotal+FP
    FNTotal=FNTotal+FN


    i=i+1

print("----Total----")
print(f"TP: {TPTotal}, TN: {TNTotal}, FP:{FPTotal},FN: {FNTotal}")      


Face Recognition

Get Number of detected faces first using the common detection chosen model

In [ ]:
#Define image to work with. It represents the real classroom photo taken with the camera
classroomFolder=os.path.join("..","ImagesDeepface","RealClassroomPhotos")

m=1
numberOfDetectedFaces=[]

#Iterate through all real classroom photos
for file in os.listdir(classroomFolder):
    camera_local_file = os.path.join("..","ImagesDeepface","RealClassroomPhotos",file)
    boundingBoxDetectedFacesNthImage=[]
    image = cv2.imread(camera_local_file)
    faces = DeepFace.extract_faces(img_path=camera_local_file, detector_backend="mtcnn")
    #Obtain images dimensions
    image_height, image_width, _ = image.shape

    #Save the coordinates of all faces in order to paint it later
    for i in range(len(faces)):
        boundingBoxDetectedFacesNthImage.append((faces[i]["facial_area"]["x"],faces[i]["facial_area"]["y"],faces[i]["facial_area"]["w"],
                                                faces[i]["facial_area"]["h"]))
    # Draw bounding boxes for each detected face using the coordinates
    for element in boundingBoxDetectedFacesNthImage:
        left = element[0]
        top = element[1]
        width = element[2]
        height =  element[3]
        cv2.rectangle(
            image, 
            (left, top), 
            (left + width, top + height), 
            (0,0,200), 
            2  
        )
    numberOfDetectedFaces.append(len(boundingBoxDetectedFacesNthImage))
    # Print number of faces detected
    print(f"{len(boundingBoxDetectedFacesNthImage)} Faces were detected in {camera_local_file}")

    # Save the annotated image in the destinated folder
    output_path =os.path.join("..","ImagesDeepface","Detection","Detectionmtcnn",f"faces_detected_{m}.jpeg")    
    cv2.imwrite(output_path, image)
    #Increase the photo's index
    m=m+1


In [ ]:

colors = [
    (255, 0, 0),     
    (0, 255, 0),   
    (0, 0, 255),    
    (255, 255, 0),  
    (255, 0, 255),  
    (0, 255, 255), 
    (128, 0, 0),  
    (0, 128, 0),   
    (0, 0, 128),   
    (128, 128, 0),  
    (128, 0, 128),  
    (0, 128, 128),  
    (64, 0, 128),   
    (128, 128, 255), 
    (0, 255, 128), 
    (255, 128, 0),   
    (128, 255, 128),
    (192, 64, 64),   
    (64, 192, 192),
    (255, 128, 192), 
    (32, 64, 255)   
]

recognition_models=["ArcFace","Facenet", "VGG-Face"]
chosen_model_index=2

photo_index=1

classroomFolder=os.path.join("..","ImagesDeepface","RealClassroomPhotos")
model=recognition_models[chosen_model_index]

boundingBoxRecognizedFaces=[]
#os.makedirs(output_folder, exist_ok=True)
m=1
for file in os.listdir(classroomFolder):
    camera_local_file=os.path.join("..","ImagesDeepface","RealClassroomPhotos",file)
    #The image read of the classroom. This will be used to redraw it
    camera_photo = cv2.imread(camera_local_file)
    recognized_counter=0
    k=0
    boundingBoxRecognizedFacesithImage=[]

    print(f"Using index {photo_index}")

    if (photo_index>=1 and photo_index<=2) or  (photo_index>=23 and photo_index<=30):
        selectedCourse="StudentsRobotics"
        student_array=["AlbaQuiroz", "AlejandroCasallas","AlejandroVelasco","AlexanderDominguez","AndresAnillo","CarlosBornacelly","DavidChaparro"
                    ,"EstebanToro","GabrielCastanez","GiohanOlivares","JesusCotes","JoseVilla", "JuanMolina","ManuelRangel","MariaBerdugo","MariaLopez",
                    "MauricioDeLaHoz", "SergioFonseca","YordiRochel"]    
    elif (photo_index>=3 and photo_index<=5) or (photo_index>=8 and photo_index<=14):
        selectedCourse="StudentsElectronicDesign1"
        student_array=["AllysonSalom", "AngieValencia","DanielDelgado","DiegoGomez", "DylanDeOro","FelipeOspino","HabidAbdala","JuanLozano","WilmanDaza"]

    elif (photo_index>=6 and photo_index<=7) or  (photo_index>=15 and photo_index<=22):
        selectedCourse="StudentsElectronicDesign2"
        student_array=["AliRada","AndreaQuintero","AndresFabregas","AndresNarvaez","BrayanRubiano","DiegoGomez","EdwinUtria","GabrielaBecerra",
                    "JuanHernandez","JuanQuintero","JuanSanchez","LucianaDeLaRosa","MariaFerrer","MayraCarreno",
                    "OctavioMorales","SamirBarcelo","SteevenBallena"]



    for student_name in student_array:
        student_photo_path=os.path.join("..","ImagesDeepface",selectedCourse,f"{student_name}.jpeg")
        result = DeepFace.verify(
                    img1_path=student_photo_path, #individual
                    img2_path=camera_local_file,
                    model_name=model,
                    detector_backend="mtcnn",
                    enforce_detection=False
                    )
        coordinatesOfGroupPhoto=result["facial_areas"]["img2"]
        x=coordinatesOfGroupPhoto["x"]
        y=coordinatesOfGroupPhoto["y"]
        h=coordinatesOfGroupPhoto["h"]
        w=coordinatesOfGroupPhoto["w"]

            # Cargar la imagen
        if camera_photo is None:
            raise FileNotFoundError(f"No se pudo cargar la imagen: {classroomFolder}")

        # Dibujar el rectángulo
        if str(result["verified"])=="True":
            x, y, w, h = coordinatesOfGroupPhoto["x"], coordinatesOfGroupPhoto["y"], coordinatesOfGroupPhoto["w"], coordinatesOfGroupPhoto["h"]
            cv2.rectangle(camera_photo, (x, y), (x + w, y + h), colors[k], 2)  
            padding = 5 
            text_box_height = 25  
            # Add text for each box
            cv2.putText(camera_photo, f"{student_name}", (x + 10, y - padding - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[k], 1)
            print(f"{student_name} was recognized in photo {m}")
            recognized_counter=recognized_counter+1

            boundingBoxRecognizedFacesithImage.append((student_name,x,y,h,w))

        else:
            print(f"{student_name} was NOT recognized in photo {m}")
            boundingBoxRecognizedFacesithImage.append((student_name,0,0,0,0))
        k=k+1
    boundingBoxRecognizedFaces.append(boundingBoxRecognizedFacesithImage)


    output_path = os.path.join("..","ImagesDeepface","Recognition",f"Recognition{model}",f"faces_in_classroom_{m}_{recognized_counter}.jpeg")
        # Guardar la imagen modificada
    cv2.imwrite(output_path, camera_photo)
    print(f"Imagen guardada en: {output_path}")
    m=m+1
    photo_index=photo_index+1

Calculate IoU

In [ ]:

def calculate_iou(box1, box2):
    """
    Calculate IoU between 2 boxes.
    box1, box2: tuples like (x, y, width, height)
    return IoU value that ranges from 0 to 1
    """
    name1,x1, y1, w1, h1 = box1
    name2, x2, y2, w2, h2 = box2
    
    x_intersect_1 = max(x1, x2)
    y_intersect_1 = max(y1, y2)
    x_intersect_2 = min(x1 + w1, x2 + w2)
    y_intersect_2 = min(y1 + h1, y2 + h2)
    
    if x_intersect_2 <= x_intersect_1 or y_intersect_2 <= y_intersect_1:
        return 0.0
    
    area_intersect = (x_intersect_2 - x_intersect_1) * (y_intersect_2 - y_intersect_1)
    area_box1 = w1 * h1
    area_box2 = w2 * h2
    
    area_union = area_box1 + area_box2 - area_intersect
    
    iou = area_intersect / area_union
    
    return iou


Define tuples from baseline (Manual correction and verificatio)

In [ ]:
with open(os.path.join("..","Data","faceGallery.txt"), "r") as file:
    content = file.read()

faceGallery=ast.literal_eval(content)


m=1
#The N-Element contains tuples of the N-Th image with informations of students to be recognized
tuples=[]
for element in faceGallery:
    i=1
    #(f"photo {m}-----------------------------------------")
    filteredFaceGalleryObject=element[next(iter(element))]["regions"]
    tuplesnthPhoto=[]
    for element in filteredFaceGalleryObject:
        #print(f"Person {i}-----")
        region=element["shape_attributes"]
        identity=element["region_attributes"]
        #print(f"x = {region["x"]} \ny = {region["y"]} \nwidth = {region["width"]}\nheight = {region["height"]} \nidentifier = {identity["name"]}")
        tuplesnthPhoto.append((identity["name"],region["x"],region["y"], region["width"], region["height"]))
        i=i+1
    tuples.append(tuplesnthPhoto)
    m=m+1


Iterate through the base line and compare it to the results obtained by Deepface's models. Then calculate TP, TN, FP, FN

In [ ]:
#It searchs based on face gallery/Base Line: It compares my definition of the existance of specific students and compares it to the found studens
#The result wouldn't be affected if the search direction is reversed

i=0
#Loop through all the photos

TPTotal=0
TNTotal=0
FPTotal=0
FNTotal=0

for photoObject in tuples:
    print(f"Results for photo {i+1}")
    TP=0
    TN=0
    FP=0
    FN=0
    #Obtain the array of faces recognized with Rekognition in that photo
    tuplesToCompare=boundingBoxRecognizedFaces[i]

    listOfNamesBaseline=[]
    for element in photoObject:
        listOfNamesBaseline.append(element[0])

    listOfNamesModel=[]
    for element in tuplesToCompare:
        listOfNamesModel.append(element[0])


    for element in photoObject:
        if(element[0] in listOfNamesModel):
            for identity in tuplesToCompare:
                IoU=calculate_iou(element,identity)
                #print(f"{element} compared with {identity}")

                #Case True Positive: Identity and bounding box match
                if(IoU>=0.4 and element[0]==identity[0]):
                    TP=TP+1

                #Case False Positive: no identity match, however, the bounding boxes are similar
                if(IoU>=0.4 and element[0]!=identity[0]):
                    FP=FP+1
                #Case True Negative: No bounding box match. However, the identity is different
                if(IoU<0.4 and element[0]!=identity[0]):
                    TN=TN+1
                #Case False Negative: The identity matches. However, bonding boxes do not correspond
                if(IoU<0.4 and element[0]==identity[0]):
                    FN=FN+1 
            TN=TN+numberOfDetectedFaces[i]-len(listOfNamesModel)
        else:
            FN=FN+1
            TN=TN+numberOfDetectedFaces[i]-1
            print(f"{element[0]} not in model's list and adds 1 FN and {len(listOfNamesBaseline)-1} TN")
    TPTotal=TPTotal+TP
    TNTotal=TNTotal+TN
    FPTotal=FPTotal+FP
    FNTotal=FNTotal+FN

    print(f"TP : {TP} \nTN : {TN} \nFP : {FP} \nFN : {FN}")
    i=i+1

print("----Total----")
print(f"TP: {TPTotal}, TN: {TNTotal}, FP:{FPTotal},FN: {FNTotal}")      


Mathematical metrics for analizing facial similarity

In [ ]:
from scipy.spatial.distance import euclidean,cosine

#Obtain the analysis of the image
embedding1=DeepFace.represent(img_path="single2.jpeg",model_name="ArcFace", detector_backend="retinaface")
embedding2=DeepFace.represent(img_path="single11.jpeg",model_name="ArcFace", detector_backend="retinaface")

#Extract the multi-dimensional vector only
embedding1Vector1=np.array(embedding1[0]["embedding"])
embedding1Vector2=embedding2[0]["embedding"]

def cosine_similarity(embedding1, embedding2):
    dot_product = np.dot(embedding1, embedding2)
    norm1 = np.linalg.norm(embedding1)
    norm2 = np.linalg.norm(embedding2)
    similarity = dot_product / (norm1 * norm2)
    return similarity

print("Cosine similarity is "+ str(cosine_similarity(embedding1Vector1,embedding1Vector2)))

def euclidean_distance(embedding1,embedding2):
    return euclidean(embedding1,embedding2)

print("Euclidean distance is "+ str(euclidean_distance(embedding1Vector1,embedding1Vector2)))

